# Topic-specific term associations through word representations
## How do Democrats and Republicans talk different about jobs

https://github.com/JasonKessler/scattertext

Cite as:
Jason S. Kessler. Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ. Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL): System Demonstrations. 2017.

Link to preprint: https://arxiv.org/abs/1703.00565

`
@article{kessler2017scattertext,
  author    = {Kessler, Jason S.},
  title     = {Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ},
  booktitle = {ACL System Demonstrations},
  year      = {2017},
}
`

In [2]:
%matplotlib inline
import scattertext as st
from gensim.models import word2vec
import re, io, itertools
from pprint import pprint
import pandas as pd
import numpy as np
import spacy.en
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [3]:
#nlp = spacy.en.English()
# If this doesn't work, please uncomment the following line and use a regex-based parser instead
nlp = st.whitespace_nlp_with_sentences

## Load the 2012 Conventions Dataset

In [4]:
convention_df = st.SampleCorpora.ConventionData2012.get_data()
convention_df['parsed'] = convention_df.text.apply(nlp)
corpus = st.CorpusFromParsedDocuments(convention_df, category_col='party', parsed_col='parsed').build()

## Use Gensim to run Word2Vec on the corpus.
### Word2Vec encodes each word in a dense K-dimensional vector space
### Cosine distances between terms vectors correspond to semantic similarity 

In [5]:
model = word2vec.Word2Vec(size=100, alpha=0.025, window=10, min_count=5, 
                          max_vocab_size=None, sample=0, seed=1, workers=1, min_alpha=0.00001, 
                          sg=1, hs=1, negative=0, cbow_mean=0, iter=1, null_word=0,
                          trim_rule=None, sorted_vocab=1)
model = st.Word2VecFromParsedCorpus(corpus, model).train()
model.wv.most_similar('tax')

[('deficit', 0.7314009666442871),
 ('cuts', 0.7105352878570557),
 ('breaks', 0.6965233087539673),
 ('increase', 0.666980504989624),
 ('code', 0.6580841541290283),
 ('wealthy', 0.651960551738739),
 ('millionaires', 0.6459797024726868),
 ('cut', 0.6393861770629883),
 ('massive', 0.6361809968948364),
 ('taxes', 0.6309477090835571)]

## Draw the Scattertext by only coloring points that have are associated with a category (p < 0.05 via log-odds w/ prior)
### The top Democratic and Republican terms are raked by their similarity to "jobs"
#### Only the terms associated to a category are considered. 
### On the far right, the most similar terms, regardless of category association, are listed.

In [8]:
target_term = 'jobs'

html = st.word_similarity_explorer_gensim(corpus,
                                          category='democrat',
                                          category_name='Democratic',
                                          not_category_name='Republican',
                                          target_term=target_term,
                                          minimum_term_frequency=5,
                                          width_in_pixels=1000,
                                          word2vec=model,
                                          metadata=convention_df['speaker'])
file_name = 'output/demo_similarity_gensim.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

## Instead of using vectors trained on the Corpus, we can use the spaCy-provided word vectors trained on the Common Crawl Corpus.
### These are trained on a lot more data, but aren't specific to the corpus

In [9]:
# Note: this will fail if you did not use spaCy as your parser.
html = st.word_similarity_explorer(corpus,
                                   category='democrat',
                                   category_name='Democratic',
                                   not_category_name='Republican',
                                   target_term='jobs',
                                   minimum_term_frequency=5,
                                   width_in_pixels=1000,
                                   metadata=convention_df['speaker'])
file_name = 'output/demo_similarity.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)